In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_ascertain as dr_asc
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_ascertain as dr_asc
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE,
        dr_w.Phases.FUN,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    # [
    #     dr_w.Phases.BASE,
    # ]
]

wesad_label_type = "stai"
asc_label_type = dr_asc.SelfReports.AROUSAL

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

threshold = "dynamic"
asc_threshold = "dynamic"
test_size = 0.8


In [3]:
# TRAIN ON ASCERTAIN AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for j, phases_wesad in enumerate(model_phases_wesad):
    print(f"WESAD PHASES {j} " + "-"*50)
    x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold)
    x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
    # drop subjects with noisy data
    inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
    x_a = x_a.drop(labels=inds, axis=0)
    y_a = y_a.drop(labels=inds, axis=0)
    
    # x = x[x['subject'] != 8.0]
    # y = y[y['subject'] != 8.0]
    
    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    # print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
    # print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

    # 0-1 scaling
    for c in range(3, len(x_a.columns)):
        data_col = x_a[x_a.columns[c]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x_a[x_a.columns[c]] = data_col
    # 0-1 scaling
    for c in range(3, len(x_b.columns)):
        data_col = x_b[x_b.columns[c]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x_b[x_b.columns[c]] = data_col

    acc_results = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
    }
    reports = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
    }
    num_iters = 10
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])

    for model_name in acc_results.keys():
        acc = np.mean(acc_results[model_name])
        print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
        if acc > 0.6:
            print(f"Model evaluation metrics for {model_name}:")
            p = np.mean([report["precision"] for report in reports[model_name]])
            r = np.mean([report["recall"] for report in reports[model_name]])
            f1 = np.mean([report["f1"] for report in reports[model_name]])
            auc = np.mean([report["auc"] for report in reports[model_name]])
            report = reports[model_name]
            print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("\n")

WESAD PHASES 0 --------------------------------------------------
y_train:
1    910
0    652
Name: label, dtype: int64
y_test:
0    34
1    21
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [24 31]
Model LogReg, Predictions: [0], [55]
Model XGB, Predictions: [0 1], [44 11]
y_train:
1    912
0    650
Name: label, dtype: int64
y_test:
0    36
1    19
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [ 5 50]
Model LogReg, Predictions: [0 1], [18 37]
Model XGB, Predictions: [0 1], [19 36]
y_train:
1    910
0    652
Name: label, dtype: int64
y_test:
0    34
1    21
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [ 8 47]
Model LogReg, Predictions: [0], [55]
Model XGB, Predictions: [0 1], [30 25]
y_train:
1    911
0    651
Name: label, dtype: int64
y_test:
0    35
1    20
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [ 7 48]
Model LogReg, Predictions: [0 1], [53  2]
Model XGB, Predictions: [0 1], [38 17]
y_train:
1    911
0    651
Name: label, dtype: int64

In [4]:
# TRAIN ON WESAD AND TEST ON ASCERTAIN
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for i, phases_wesad in enumerate(model_phases_wesad):
        print(f"WESAD PHASES {i} " + "-"*50)
        x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
        x_b, y_b = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold)
        # drop subjects with noisy data
        inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
        x_b = x_b.drop(labels=inds, axis=0)
        y_b = y_b.drop(labels=inds, axis=0)
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500
        y_b["subject"] = y_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
        print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

        acc_results = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        reports = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.6:
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Report:\nPrecision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

WESAD PHASES 0 --------------------------------------------------
y_a:
0    44
1    26
Name: label, dtype: int64
y_b:
1    905
0    642
Name: label, dtype: int64
y_train:
1    210
0    171
Name: label, dtype: int64
y_test:
1    721
0    515
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [   9 1227]
Model LogReg, Predictions: [0 1], [  75 1161]
Model XGB, Predictions: [0 1], [468 768]
y_train:
1    219
0    208
Name: label, dtype: int64
y_test:
1    712
0    478
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [   3 1187]
Model LogReg, Predictions: [0 1], [  25 1165]
Model XGB, Predictions: [0 1], [489 701]
y_train:
1    219
0    180
Name: label, dtype: int64
y_test:
1    712
0    506
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [   4 1214]
Model LogReg, Predictions: [0 1], [  67 1151]
Model XGB, Predictions: [0 1], [554 664]
y_train:
1    249
0    192
Name: label, dtype: int64
y_test:
1    682
0    494
Name: label, dtype: int64
Model SVM, Predictions: [1],